### API-without-secrets
- In Vulkan, a render pass represents (or describes) a set of framebuffer attachments (images) required for drawing operations and a collection of subpasses that drawing operations will be ordered into. It is a construct that collects all color, depth and stencil attachments and operations modifying them in such a way that driver does not have to deduce this information by itself what may give substantial optimization opportunities on some GPUs. A subpass consists of drawing operations that use (more or less) the same attachments. Each of these drawing operations may read from some input attachments and render data into some other (color, depth, stencil) attachments. A render pass also describes the dependencies between these attachments: in one subpass we perform rendering into the texture, but in another this texture will be used as a source of data (that is, it will be sampled from). All this data help the graphics hardware optimize drawing operations.

- Tiling defines the inner memory structure of an image, Images may have linear or optimal tiling (buffers always have linear tiling). 
- Images with linear tiling have their texels laid out linearly, one texel after another, one row after another, and so on. - -- When we specify an optimal tiling for images, it means that we don’t know how their memory is structured.
- That’s why it is strongly suggested to always specify optimal tiling for images.
- Layout, as it was described in a tutorial about swapchains, defines an image’s memory layout and is strictly connected with the way in which we want to use an image. Each specific usage has its own memory layout. Before we can use an image in a given way we need to perform a layout transition.

- When we want to change the way in which an image is used, we need to perform the above-mentioned layout transition. We must specify a current (old) layout and a new one. The old layout can have one of two values: current image layout or an undefined layout. When we specify the value of a current image’s layout, the image contents are preserved during transition. But when we don’t need an image’s contents, we can provide an undefined layout. In this way layout transition may be performed faster.

- We want to use an image as a texture inside shaders. For this purpose we specify the VK_IMAGE_USAGE_SAMPLED_BIT usage. We also need a way to upload data to the image. We are going to read it from an image file and copy it to the image object. This can be done by transferring data using a staging resource. In such a case, the image will be a target of a transfer operation; that’s why we also specify the VK_IMAGE_USAGE_TRANSFER_DST_BIT usage.

- Of course, when we want to bind a memory to an image, we don’t need to create a new memory object each time. It is more optimal to create a small number of larger memory objects and bind parts of them by providing a proper offset value.

- The operation of copying data from a buffer to an image requires recording a command buffer and submitting it to a queue.

- One last thing is to perform another layout transition. Our image will be used as a texture inside shaders, so we need to transition it to a VK_IMAGE_LAYOUT_SHADER_READ_ONLY_OPTIMAL layout. After that, we can end our command buffer, submit it to a queue, and wait for the transfer to complete (in a real-life application, we should skip waiting and synchronize operations in some other way; for example, using semaphores, to avoid unnecessary pipeline stalls).